In [1]:
import pandas as pd
import geopandas as gpd
import re
import plotly as pl
import plotly.express as px

In [2]:
MKE_CensusTract_csv = pd.read_csv("DataFiles/MKE_CensusTract_2023.csv")

In [41]:
def CensusTract_Reformat(fresh_csv):
    #first step is to reduce the columns, in this study we are just focusing on total and ignoring margin of error
    csv_col_names = list(fresh_csv.columns)
    kept_cols = []
    for i in range(len(csv_col_names)):
        #filter
        if 'Total' in csv_col_names[i] and 'Estimate' in csv_col_names[i]:
            kept_cols.append(csv_col_names[i])
        #the 'label (Grouping)' will eventually be used as the index
        elif 'Label (Grouping)' == csv_col_names[i]:
            kept_cols.append(csv_col_names[i])
        else:
            pass
    CensusTract = fresh_csv[kept_cols]
    #second step is to rename the eventual index names
    for row in range(len(CensusTract['Label (Grouping)'])):
        if '\xa0' in CensusTract['Label (Grouping)'][row]:
            CensusTract.loc[row, 'Label (Grouping)'] = CensusTract['Label (Grouping)'][row].split('\xa0')[-1]
        else:
            pass
    #third step is to rename the columns to reduce them to just the census tract numbers to be easily joined with the geojson file
    fullclean_col_names = list(CensusTract.columns)
    for column in fullclean_col_names:
        new_col_name = ''
        if column == 'Label (Grouping)':
            pass
        elif '.' in column:
            new_col_name = column.split(';')[0].split(' ')[2].replace('.','')
            CensusTract.rename(columns={column : int(new_col_name)},inplace = True)
        elif ';' in column:
            new_col_name = column.split(';')[0].split(' ')[2]
            if len(new_col_name) > 4:
                pass
            else:
                new_col_name = new_col_name + '00'
            CensusTract.rename(columns={column : int(new_col_name)},inplace = True)
        else:
            print(type(column))
    #Fourth step is to convert all the strings to integers
    for col in list(CensusTract.columns):
        if col == 'Label (Grouping)':
            pass
        else:
            CensusTract[col] = CensusTract[col].str.replace(',','')
            CensusTract[col] = pd.to_numeric(CensusTract[col], errors = 'ignore')
    #Final step is turn 'label (grouping)' into the index and return the dataframe
    return CensusTract.set_index('Label (Grouping)')

In [24]:
CensusTract_Subsets = ['Age','Sex','Race','Nativity','Marital','Education','Income','Poverty','Housing']
def CensusTract_Subset(cleaned_csv, subset):
    if subset == 'Age':
        return cleaned_csv.loc[['Population 1 year and over','1 to 4 years', '5 to 17 years', '18 to 24 years', '25 to 34 years', '35 to 44 years', 
                                '45 to 54 years', '55 to 64 years','65 to 74 years', '75 years and over']]
    elif subset == 'Sex':
        return cleaned_csv.loc[['Male','Female']]
    elif subset == 'Race':
        return cleaned_csv.loc[['One race', 'White', 'Black or African American', 'American Indian and Alaska Native', 'Asian', 'Native Hawaiian and Other Pacific Islander',
                                'Some other race', 'Two or more races', 'Hispanic or Latino origin (of any race)','White alone, not Hispanic or Latino']]
    elif subset == 'Nativity':
        return cleaned_csv.loc[['Native', 'Foreign born', 'Naturalized U.S. citizen', 'Not a U.S. citizen']]
    elif subset == 'Marital':
        return cleaned_csv.loc[['Population 15 years and over', 'Never married', 'Now married, except separated','Divorced or separated','Widowed',]]
    elif subset == 'Education':
        return cleaned_csv.loc[['Population 25 years and over', 'Less than high school graduate', 'High school graduate (includes equivalency)', 
                                "Some college or associate's degree","Bachelor's degree", 'Graduate or professional degree',]]
    elif subset == 'Income':
        return cleaned_csv.loc[['Population 15 years and over', '$1 to $9,999 or loss', '$10,000 to $14,999', '$15,000 to $24,999', '$25,000 to $34,999',
                                '$35,000 to $49,999','$50,000 to $64,999', '$65,000 to $74,999', '$75,000 or more', 'Median income (dollars)']]
    elif subset == 'Poverty':
        return cleaned_csv.loc[['Population 1 year and over for whom poverty status is determined', 'Below 100 percent of the poverty level',
                                '100 to 149 percent of the poverty level', 'At or above 150 percent of the poverty level',]]
    elif subset == 'Housing':
        return cleaned_csv.loc[['Population 1 year and over in housing units','Householder lived in owner-occupied housing units', 'Householder lived in renter-occupied housing units']]
    else:
        return "Acceptable subsets are: " + str(CensusTract_Subsets)

In [43]:
dataset = CensusTract_Reformat(MKE_CensusTract_csv)
dataset.to_csv("MKE_Cleaned.csv")

C:\Users\Greys\AppData\Local\Temp\ipykernel_153756\1436329098.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CensusTract.loc[row, 'Label (Grouping)'] = CensusTract['Label (Grouping)'][row].split('\xa0')[-1]
C:\Users\Greys\AppData\Local\Temp\ipykernel_153756\1436329098.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CensusTract.rename(columns={column : int(new_col_name)},inplace = True)
C:\Users\Greys\AppData\Local\Temp\ipykernel_153756\1436329098.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

In [32]:
dataset

,101,102,201,202,301,302,303,304,400,501,...,186500,186600,186800,186900,187000,187200,187300,187400,980000,990000
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Population 1 year and over,4695,3515,5534,6007,1256,2875,1604,3721,2731,4081,...,2177,2035,1425,2285,3536,4594,6853,3601,0,0
AGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 to 4 years,370,177,584,654,43,285,138,226,212,321,...,9,132,68,17,78,111,311,20,0,0
5 to 17 years,1323,677,1672,1463,179,881,311,881,557,962,...,250,155,228,0,228,617,1536,15,0,0
18 to 24 years,331,88,460,260,27,262,148,176,93,392,...,281,348,58,192,725,423,370,290,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Population 1 year and over in housing units,4544,3359,5497,5990,1235,2867,1603,3702,2728,4065,...,2138,2035,1304,2113,3449,4094,6824,3540,0,0
Householder lived in owner-occupied housing units,1203,1610,933,3383,1171,875,845,3535,726,1345,...,406,509,29,767,605,3443,6618,1067,0,0
Householder lived in renter-occupied housing units,3341,1749,4564,2607,64,1992,758,167,2002,2720,...,1732,1526,1275,1346,2844,651,206,2473,0,0


In [ ]:
geo = gpd.read_file('DataFiles/MKE_CensusTract.geojson')
geo